In [1]:
import pandas as pd
from pathlib import Path
import requests
import json
import time
import numpy as np
import time

from process_inat_data import format_inat_data
from create_normalized_taxa import create_taxon, get_row_ranks, all_ranks, main_ranks

In [2]:
def log_df(df, nrows=5):
    print(df.shape)
    return df.head(nrows)

In [3]:
def print_json(obj):
    print(json.dumps(obj,  indent=4))

In [4]:
all_taxa =  Path('outputs', 'combine_taxa_list.csv')
all_taxa_inat =  Path('outputs', 'combine_taxa_list_inat_data.csv')
de_taxa =  Path('outputs', 'de_taxa_list.csv')


# create combine taxa list

create one taxa file from all the project observations files and LA indicator list

In [46]:
existing_df = pd.read_csv(all_taxa, dtype=str )
existing_df['has_inat_data'] = True
existing_df.shape

(3851, 5)

In [47]:
dfs = []

taxa_cols = [ 
    'scientific_name', 
    'common_name',
    'iconic_taxon_name', 
    'taxon_id', 
]

for file_path in Path().glob('../data/**/observations*.csv'):
    df = pd.read_csv(file_path,  usecols=taxa_cols, dtype=str)
    df = df.dropna(subset=['taxon_id'])
    df.rename(columns=new_cols, inplace=True)
    dfs.append(df)


indicator_cols =  [ 
    'scientific_name', 
    'common_name',
    'iconic_taxon_name', 
    'taxon_id', 
]    
file = '../data/los-angeles-bioblitz/indicator_species_with_inat.csv'
df = pd.read_csv(file,  dtype=str, usecols=indicator_cols)
dfs.append(df)
    
    
combine_df = pd.concat(dfs)
combine_df = combine_df.drop_duplicates(subset='taxon_id')

log_df(combine_df)
 

(4752, 4)


,scientific_name,common_name,iconic_taxon_name,taxon_id
0,Physalia physalis,Portuguese Man o' War,Animalia,117302
3,Aurelia marginalis,Southern Moon Jelly,Animalia,986245
6,Chrysaora colorata,Purple-striped Sea Nettle,Animalia,69839
7,Chrysaora fuscescens,Pacific Sea Nettle,Animalia,48479
10,Chrysaora achlyos,Black Jelly,Animalia,83415


In [48]:
combine_df = combine_df.merge(existing_df, how='left')
combine_df.to_csv(all_taxa, index=False)

# add inat data to combine taxa list

get info from inat API for each taxa without inat data

get existing taxa inat data

In [97]:
inat_df = pd.read_csv(all_taxa_inat, dtype=str)
log_df(inat_df)

(3851, 336)


,scientific_name,common_name,iconic_taxon_name,taxon_id,kingdom,phylum,class,order,family,genus,...,genushybrid_id,genushybrid_wikipedia_url,genushybrid_iconic_taxon_name,genushybrid_common_name,genushybrid_parent_id,genushybrid_ancestor_ids,genushybrid,genushybrid_photo_url,genushybrid_photo_attribution,genushybrid_photo_license_code
0,Deidamia inscriptum,Lettered Sphinx,Insecta,143452,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acer,maples,Plantae,47727,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Plantago lanceolata,ribwort plantain,Plantae,53178,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Poa pratensis,Kentucky bluegrass,Plantae,60307,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Magnoliopsida,dicots,Plantae,47124,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


get taxa without inat data

In [98]:
taxa_df = pd.read_csv(all_taxa, dtype=str)
taxa_df = taxa_df[taxa_df['has_inat_data'].isna()]
log_df(taxa_df)

(901, 5)


,scientific_name,common_name,iconic_taxon_name,taxon_id,has_inat_data
0,Physalia physalis,Portuguese Man o' War,Animalia,117302,NaN
1,Aurelia marginalis,Southern Moon Jelly,Animalia,986245,NaN
2,Chrysaora colorata,Purple-striped Sea Nettle,Animalia,69839,NaN
3,Chrysaora fuscescens,Pacific Sea Nettle,Animalia,48479,NaN
4,Chrysaora achlyos,Black Jelly,Animalia,83415,NaN


break up taxon_ids into strings with multiple ids to send multiple ids per api call

In [164]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    # https://stackoverflow.com/a/312464
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

ids = []
for index, row in taxa_df.iterrows():
    ids.append(str(row['taxon_id']))  
    
id_strings = [','.join(chunk) for chunk in  list(chunks(ids, 20))]

id_strings[0:2]

['117302,986245,69839,48479,83415,256089,59698,49601,49604,41553,445671,370954,540910,432549,86896,604230,154711,128144,87507,48921',
 '496338,360314,199347,109865,59683,48778,154710,48478,295349,50498,565367,861465,254081,387555,973000,260959,35164,121639,195134,48329']

In [166]:
for index, id_string in enumerate(id_strings):
    time.sleep(2)
    print(index, end=' ')
    url = f'https://api.inaturalist.org/v1/taxa/{id_string}'
    
    response = requests.get(url)
    if response.status_code == 200:
        for json_data in response.json()['results']:
            id = json_data['id']
            result  = format_inat_data(json_data)
            for col in result:
                taxa_df.loc[taxa_df['taxon_id'] == str(id), col] = result[col]
    else:
        print('bad', url)
   

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 

In [168]:
log_df(taxa_df)

(901, 336)


,scientific_name,common_name,iconic_taxon_name,taxon_id,kingdom,phylum,class,order,family,genus,...,genushybrid_id,genushybrid_wikipedia_url,genushybrid_iconic_taxon_name,genushybrid_common_name,genushybrid_parent_id,genushybrid_ancestor_ids,genushybrid,genushybrid_photo_url,genushybrid_photo_attribution,genushybrid_photo_license_code
0,Physalia physalis,Portuguese Man o' War,Animalia,117302,Animalia,Cnidaria,Hydrozoa,Siphonophorae,Physaliidae,Physalia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aurelia marginalis,Southern Moon Jelly,Animalia,986245,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Ulmaridae,Aurelia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chrysaora colorata,Purple-striped Sea Nettle,Animalia,69839,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chrysaora fuscescens,Pacific Sea Nettle,Animalia,48479,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chrysaora achlyos,Black Jelly,Animalia,83415,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


make sure taxa_df has same columns as inat_df

In [169]:
for col in set(inat_df.columns) - set(taxa_df.columns):
    taxa_df[col] = np.nan

taxa_df  = taxa_df[inat_df.columns]    
log_df(taxa_df)

(901, 336)


,scientific_name,common_name,iconic_taxon_name,taxon_id,kingdom,phylum,class,order,family,genus,...,genushybrid_id,genushybrid_wikipedia_url,genushybrid_iconic_taxon_name,genushybrid_common_name,genushybrid_parent_id,genushybrid_ancestor_ids,genushybrid,genushybrid_photo_url,genushybrid_photo_attribution,genushybrid_photo_license_code
0,Physalia physalis,Portuguese Man o' War,Animalia,117302,Animalia,Cnidaria,Hydrozoa,Siphonophorae,Physaliidae,Physalia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aurelia marginalis,Southern Moon Jelly,Animalia,986245,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Ulmaridae,Aurelia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chrysaora colorata,Purple-striped Sea Nettle,Animalia,69839,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chrysaora fuscescens,Pacific Sea Nettle,Animalia,48479,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chrysaora achlyos,Black Jelly,Animalia,83415,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
set(taxa_df.columns) - set(inat_df.columns)

set()

In [171]:
all_inat_df = pd.concat([taxa_df, inat_df])
log_df(all_inat_df)

(4752, 336)


,scientific_name,common_name,iconic_taxon_name,taxon_id,kingdom,phylum,class,order,family,genus,...,genushybrid_id,genushybrid_wikipedia_url,genushybrid_iconic_taxon_name,genushybrid_common_name,genushybrid_parent_id,genushybrid_ancestor_ids,genushybrid,genushybrid_photo_url,genushybrid_photo_attribution,genushybrid_photo_license_code
0,Physalia physalis,Portuguese Man o' War,Animalia,117302,Animalia,Cnidaria,Hydrozoa,Siphonophorae,Physaliidae,Physalia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aurelia marginalis,Southern Moon Jelly,Animalia,986245,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Ulmaridae,Aurelia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chrysaora colorata,Purple-striped Sea Nettle,Animalia,69839,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chrysaora fuscescens,Pacific Sea Nettle,Animalia,48479,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chrysaora achlyos,Black Jelly,Animalia,83415,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
all_inat_df.to_csv(all_taxa_inat, index=False)

## browse data

In [8]:
inat_df = pd.read_csv(all_taxa_inat, dtype=str)

In [9]:
inat_df[inat_df['rank'] == 'genus'].shape

(776, 336)

some names are used multiple times for different organisisms

In [10]:
len(inat_df['genus'].dropna().unique())

1864

In [11]:
len(inat_df['genus_id'].dropna().unique())

1867

# normalize taxa list

create a taxa list  with separate records for all higher level taxa

In [174]:
inat_df = pd.read_csv(all_taxa_inat, dtype=str)
log_df(inat_df)

(4752, 336)


,scientific_name,common_name,iconic_taxon_name,taxon_id,kingdom,phylum,class,order,family,genus,...,genushybrid_id,genushybrid_wikipedia_url,genushybrid_iconic_taxon_name,genushybrid_common_name,genushybrid_parent_id,genushybrid_ancestor_ids,genushybrid,genushybrid_photo_url,genushybrid_photo_attribution,genushybrid_photo_license_code
0,Physalia physalis,Portuguese Man o' War,Animalia,117302,Animalia,Cnidaria,Hydrozoa,Siphonophorae,Physaliidae,Physalia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aurelia marginalis,Southern Moon Jelly,Animalia,986245,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Ulmaridae,Aurelia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chrysaora colorata,Purple-striped Sea Nettle,Animalia,69839,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chrysaora fuscescens,Pacific Sea Nettle,Animalia,48479,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chrysaora achlyos,Black Jelly,Animalia,83415,Animalia,Cnidaria,Scyphozoa,Semaeostomeae,Pelagiidae,Chrysaora,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
data = []
for index, row in inat_df.iterrows():
    row_ranks = get_row_ranks(row, all_ranks, 'ancestor_ids', 'taxon_id')
    allowed_ranks = main_ranks + [row['rank']]
    rank_history = []
    
    for rank in row_ranks:
        if rank in allowed_ranks:
            rank_history.append(rank)
            taxon = create_taxon(row, rank, rank_history)
            data.append(taxon)


In [176]:
taxa_df = pd.DataFrame(data)
taxa_df.drop_duplicates(subset=['taxon_id'], inplace=True)

log_df(taxa_df, 8)

(7315, 6)


,taxon_id,common_name,scientific_name,rank,parent_id,taxon_ids
0,1,Animals,Animalia,kingdom,48460,1
1,47534,Cnidarians,Cnidaria,phylum,1,1|47534
2,48921,Hydrozoans,Hydrozoa,class,47534,1|47534|48921
3,152823,Siphonophores,Siphonophorae,order,48921,1|47534|48921|152823
4,117304,Man O' Wars,Physaliidae,family,152823,1|47534|48921|152823|117304
5,117305,Man O' Wars,Physalia,genus,117304,1|47534|48921|152823|117304|117305
6,117302,Portuguese Man o' War,Physalia physalis,species,117305,1|47534|48921|152823|117304|117305|117302
9,48332,True Jellies,Scyphozoa,class,47534,1|47534|48332


In [177]:
print(len(inat_df['rank'].unique()))
print(len(taxa_df['rank'].unique()))

set(inat_df['rank'].unique()) - set(all_ranks)

28
28


set()

In [178]:
print(len(set(inat_df['taxon_id']) - set(taxa_df['taxon_id'])))
len(set(taxa_df['taxon_id']) - set(inat_df['taxon_id']))

0


2563

In [179]:
all_ids = set()
for rank in inat_df['rank'].unique():
    if rank in main_ranks:
        all_ids.update([id for id in inat_df[rank + '_id'].values if pd.notna(id) ])
all_ids.update(inat_df['taxon_id'])
    
print(len(all_ids))
print(len(taxa_df['taxon_id']))

print(len(all_ids - set(taxa_df['taxon_id'])))
print(len(set(taxa_df['taxon_id']) - all_ids))

7315
7315
0
0


In [180]:
taxa_df.to_csv(de_taxa, index=False)